In [1]:
import re
import glob

# data dir
data_dir = './traffic_anomalies_data/'
out_dir = './temp/'
file_number = 0

for original_file in glob.glob(f'{data_dir}*.TXT'):
    with open(original_file, 'r') as file:
        content = file.read()
    file_number += 1
    rm_strings = ['SAS Output', 'Minnesota Department of Transportation', 'Continuous Count Station Edited Data for', 
                  'Hour Ending At:', '* - Day Contains Some or All Estimated Data Due to Missing Data or Malfunctions at the ATR.', 
                  'DATE         DAY       1 am  2 am  3 am  4 am 5 am  6 am  7 am  8 am  9 am 10 am 11 am  Noon  1 pm  2 pm  3 pm  4 pm  5 pm  6 pm  7 pm  8 pm  9 pm 10 pm 11 pm 12 mid TOTAL', 
                  'DATE         DAY       HR1   HR2   HR3   HR4  HR5   HR6   HR7   HR8   HR9  HR10  HR11  HR12  HR13  HR14  HR15  HR16  HR17  HR18  HR19  HR20  HR21  HR22  HR23  HR24 TOTAL']
    for rm_str in rm_strings:
        content = content.replace(rm_str, '')
    # remove redundant spaces
    content = re.sub(r' +', ' ', content)
    # remove redundant newlines
    content = re.sub(r'\n+', '\n', content)
    # remove dashes
    content = re.sub(r'-+', '', content)
    # fix rows with leading space
    content = re.sub(r'\n ', '\n', content)
    # fix double newlines
    content = re.sub(r'\n\n', '\n', content)
    # remove *
    content = re.sub(r'\* ', '', content)

    # months
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    directions = ['East', 'West']
    for month in months:
        for direction in directions:
            try:
                table = re.search(f'Station 301, Direction {direction}, {month}([\s\S]*?)AVERAGES', content).group(1)
                # strip leading and trailing whitespace
                table = table.strip()
                file_name = f'{out_dir}{file_number}_{direction}_{month}.txt'
                with open(file_name, 'w') as text_file:
                    text_file.write(table)
            except AttributeError:
                print(f'Station 301, Direction {direction}, {month}([\s\S]*?)AVERAGES')
                print('FAILED')

Station 301, Direction East, October([\s\S]*?)AVERAGES
FAILED
Station 301, Direction West, October([\s\S]*?)AVERAGES
FAILED
Station 301, Direction East, November([\s\S]*?)AVERAGES
FAILED
Station 301, Direction West, November([\s\S]*?)AVERAGES
FAILED
Station 301, Direction East, December([\s\S]*?)AVERAGES
FAILED
Station 301, Direction West, December([\s\S]*?)AVERAGES
FAILED


In [2]:
import pandas as pd


directions = ['East', 'West']
for direction in directions:
    master_df = pd.DataFrame()
    for original_file in glob.glob(f'{out_dir}*{direction}*.txt'):
        print(original_file)
        df = pd.read_csv(original_file, sep=' ', header=None)
        print(df.shape)
        print(master_df.shape)
        master_df = pd.concat([master_df, df])
        print(master_df.shape)
    # rename the columns
    ew = direction[0]
    # shifted the time from the official data by 1 hour so it 'starts at' the following time
    master_df.columns = ['Month', 'Day', 'Year', 'Week_Day', '00:00', '01:00', '02:00', '03:00', '04:00', '05:00', 
                         '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', 
                         '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', 
                         '20:00', '21:00', '22:00', '23:00', 'Total']
    # combine the first three into a date
    master_df['Date'] = master_df['Month'] + ' ' + master_df['Day'].astype('str') + ' ' + master_df['Year'].astype('str')
    master_df['Date'] = pd.to_datetime(master_df['Date'], format='%b %d, %Y')
    del master_df['Month'] 
    del master_df['Day']
    del master_df['Year']
    del master_df['Week_Day']
    master_df['Date'] = pd.to_datetime(master_df['Date'])
    master_df['Day_of_Week'] = master_df['Date'].dt.dayofweek
    master_df['Year'] = master_df['Date'].dt.year
    master_df['Month'] = master_df['Date'].dt.month
    master_df['Week_of_Year'] = master_df['Date'].dt.week
    master_df['Day_of_Month'] = master_df['Date'].dt.day
    master_df.to_csv(f'Station_301_{direction}.csv.gz', index=False, compression='gzip')

./temp\10_East_April.txt
(30, 29)
(0, 0)
(30, 29)
./temp\10_East_August.txt
(31, 29)
(30, 29)
(61, 29)
./temp\10_East_December.txt
(31, 29)
(61, 29)
(92, 29)
./temp\10_East_February.txt
(28, 29)
(92, 29)
(120, 29)
./temp\10_East_January.txt
(31, 29)
(120, 29)
(151, 29)
./temp\10_East_July.txt
(31, 29)
(151, 29)
(182, 29)
./temp\10_East_June.txt
(30, 29)
(182, 29)
(212, 29)
./temp\10_East_March.txt
(31, 29)
(212, 29)
(243, 29)
./temp\10_East_May.txt
(31, 29)
(243, 29)
(274, 29)
./temp\10_East_November.txt
(30, 29)
(274, 29)
(304, 29)
./temp\10_East_October.txt
(31, 29)
(304, 29)
(335, 29)
./temp\10_East_September.txt
(30, 29)
(335, 29)
(365, 29)
./temp\11_East_April.txt
(30, 29)
(365, 29)
(395, 29)
./temp\11_East_August.txt
(31, 29)
(395, 29)
(426, 29)
./temp\11_East_December.txt
(31, 29)
(426, 29)
(457, 29)
./temp\11_East_February.txt
(29, 29)
(457, 29)
(486, 29)
./temp\11_East_January.txt
(31, 29)
(486, 29)
(517, 29)
./temp\11_East_July.txt
(31, 29)
(517, 29)
(548, 29)
./temp\11_East_

(4595, 29)
./temp\5_East_October.txt
(31, 29)
(4595, 29)
(4626, 29)
./temp\5_East_September.txt
(30, 29)
(4626, 29)
(4656, 29)
./temp\6_East_April.txt
(30, 29)
(4656, 29)
(4686, 29)
./temp\6_East_August.txt
(31, 29)
(4686, 29)
(4717, 29)
./temp\6_East_December.txt
(31, 29)
(4717, 29)
(4748, 29)
./temp\6_East_February.txt
(28, 29)
(4748, 29)
(4776, 29)
./temp\6_East_January.txt
(31, 29)
(4776, 29)
(4807, 29)
./temp\6_East_July.txt
(31, 29)
(4807, 29)
(4838, 29)
./temp\6_East_June.txt
(30, 29)
(4838, 29)
(4868, 29)
./temp\6_East_March.txt
(31, 29)
(4868, 29)
(4899, 29)
./temp\6_East_May.txt
(31, 29)
(4899, 29)
(4930, 29)
./temp\6_East_November.txt
(30, 29)
(4930, 29)
(4960, 29)
./temp\6_East_October.txt
(31, 29)
(4960, 29)
(4991, 29)
./temp\6_East_September.txt
(30, 29)
(4991, 29)
(5021, 29)
./temp\7_East_April.txt
(30, 29)
(5021, 29)
(5051, 29)
./temp\7_East_August.txt
(31, 29)
(5051, 29)
(5082, 29)
./temp\7_East_December.txt
(31, 29)
(5082, 29)
(5113, 29)
./temp\7_East_February.txt
(29

(31, 29)
(2981, 29)
(3012, 29)
./temp\1_West_June.txt
(30, 29)
(3012, 29)
(3042, 29)
./temp\1_West_March.txt
(31, 29)
(3042, 29)
(3073, 29)
./temp\1_West_May.txt
(31, 29)
(3073, 29)
(3104, 29)
./temp\1_West_November.txt
(30, 29)
(3104, 29)
(3134, 29)
./temp\1_West_October.txt
(31, 29)
(3134, 29)
(3165, 29)
./temp\1_West_September.txt
(30, 29)
(3165, 29)
(3195, 29)
./temp\2_West_April.txt
(30, 29)
(3195, 29)
(3225, 29)
./temp\2_West_August.txt
(31, 29)
(3225, 29)
(3256, 29)
./temp\2_West_December.txt
(31, 29)
(3256, 29)
(3287, 29)
./temp\2_West_February.txt
(28, 29)
(3287, 29)
(3315, 29)
./temp\2_West_January.txt
(31, 29)
(3315, 29)
(3346, 29)
./temp\2_West_July.txt
(31, 29)
(3346, 29)
(3377, 29)
./temp\2_West_June.txt
(30, 29)
(3377, 29)
(3407, 29)
./temp\2_West_March.txt
(31, 29)
(3407, 29)
(3438, 29)
./temp\2_West_May.txt
(31, 29)
(3438, 29)
(3469, 29)
./temp\2_West_November.txt
(30, 29)
(3469, 29)
(3499, 29)
./temp\2_West_October.txt
(31, 29)
(3499, 29)
(3530, 29)
./temp\2_West_Sept

In [3]:
# let's try melting the data into a single continuous time series
melt = pd.melt(master_df, id_vars=['Date'], value_vars=['01:00', '02:00', '03:00', '04:00', '05:00', 
                         '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', 
                         '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', 
                         '20:00', '21:00', '22:00', '23:00', '00:00'], var_name='Hour', value_name='Traffic_Volume')

In [4]:
melt['ds'] = melt['Date'].astype('str') + ' ' + melt['Hour']
melt['ds'] = pd.to_datetime(melt['ds'])
del melt['Date']
del melt['Hour']

In [5]:
melt.sort_values(by=['ds'], inplace=True)
melt.set_index('ds', inplace=True)

In [6]:
melt.to_csv('ts_traffic.csv')

In [43]:
import pandas as pd

# wrangle weather data
weather = pd.read_csv('./supplemental_data/weather.csv')
weather['ds'] = pd.to_datetime(weather['ds'], format='%Y-%m-%d %H:%M:%S +0000 UTC')
weather.fillna(0, inplace=True)

In [44]:
from pytz import all_timezones

# Set starting time zone
weather['ds'] = weather['ds'].dt.tz_localize('UTC')
# convert to CST
weather['ds'] = weather['ds'].dt.tz_convert('US/Central')

In [45]:
weather.to_csv('weather_cst.csv', index=False)